# Flipkart EDA assignment

## TODO: Upload csv file before moving to next

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Flipkart Data Engineering").getOrCreate()
file_path = '/FileStore/tables/Flipkart.csv'
flipkart_df = spark.read.csv(file_path, header=True, inferSchema=True)

In [0]:
flipkart_df.printSchema()
flipkart_df.count()

root
 |-- id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- maincateg: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- actprice1: integer (nullable = true)
 |-- norating1: integer (nullable = true)
 |-- noreviews1: integer (nullable = true)
 |-- star_5f: integer (nullable = true)
 |-- star_4f: integer (nullable = true)
 |-- star_3f: integer (nullable = true)
 |-- star_2f: integer (nullable = true)
 |-- star_1f: integer (nullable = true)
 |-- fulfilled1: integer (nullable = true)

Out[2]: 5244

In [0]:
flipkart_df.show(5)

+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|   id|               title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+-----+--------------------+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| 2242|Casuals For Men  ...|   3.8|      Men|Flipkart|      999|    27928|      3543|  14238|   4295|   3457|   1962|   3976|         1|
|20532|Women Black Flats...|   3.9|    Women|Flipkart|      499|     3015|       404|   1458|    657|    397|    182|    321|         1|
|10648|Women Gold Wedges...|   3.9|    Women|Flipkart|      999|      449|        52|    229|     70|     71|     33|     46|         1|
|20677|Men's Height Incr...|   3.9|      Men|Flipkart|     2999|      290|        40|    141|     51|     49|     17|     32|         1|
|12593|Loafers For Men  ...|   3.9|      

In [0]:
# Display all category names
flipkart_df.select("maincateg").distinct().show()


+---------+
|maincateg|
+---------+
|     null|
|      Men|
|    Women|
+---------+



In [0]:
# Filter products with rating > 4.5 and more than 1000 reviews
highly_rated_df = flipkart_df.filter((flipkart_df["Rating"] > 4) & (flipkart_df["noreviews1"] > 100)).select("id", "title", "Rating", "noreviews1").show()


+-----+--------------------+------+----------+
|   id|               title|Rating|noreviews1|
+-----+--------------------+------+----------+
|13859|Women Black Wedge...|   4.2|       487|
|19453|Denill Ankle Leng...|   4.1|       961|
| 3205|     SFG-23 Slippers|   4.1|      1598|
| 7625|White casual, gym...|   4.2|       381|
|17791|Synthetic Leather...|   4.2|       182|
|15762|Running Shoes For...|   4.1|      8254|
|15247|SM-484 Casuals Fo...|   4.2|       436|
|20247|Women Black Heels...|   4.1|       182|
|10655|Women Brown Wedge...|   4.1|       178|
|20918|GO WALK 5-SPARROW...|   4.3|       106|
|16193|crocs Unisex's Ra...|   4.3|       661|
| 8485|Men Grey Clogs Sa...|   4.1|       328|
| 7561|Men Navy Sports S...|   4.2|      2103|
| 9249|Women Black Wedge...|   4.2|       487|
|20597|Sneakers For Men ...|   4.1|       597|
|11078|Men Black Clogs S...|   4.2|      1098|
|13151|Walking Shoes For...|   4.1|       938|
|15716|Mojaris For Men  ...|   4.1|       106|
|20324|Women 

In [0]:
# Display Products in 'Men' category that are fulfilled

men_fulfilled = flipkart_df.filter((flipkart_df["maincateg"] == "Men") & (flipkart_df["fulfilled1"] == 1)).select("id", "title", "Rating", "fulfilled1").show()

+-----+--------------------+------+----------+
|   id|               title|Rating|fulfilled1|
+-----+--------------------+------+----------+
| 2242|Casuals For Men  ...|   3.8|         1|
|20677|Men's Height Incr...|   3.9|         1|
|10680|Combo Pack Of 2 C...|   3.9|         1|
|11579|MOCCASSIN Slip On...|   3.8|         1|
|12260|Long OfficeWear C...|   3.9|         1|
| 3205|     SFG-23 Slippers|   4.1|         1|
|14489|Dark Brown Leathe...|   3.9|         1|
|10206|Long OfficeWear C...|   3.9|         1|
| 8920|Monk Strap For Me...|   4.2|         1|
| 9762|    Men Brown Sandal|   4.6|         1|
|14490|Men Hawaii Flip F...|   3.7|         1|
|15546|Men Blue Clogs Sa...|   3.7|         1|
|10068|Java 2 IDP Flip F...|   4.1|         1|
| 6052|Derby For Men  (B...|   2.8|         1|
|11469|Combo pack of 2 s...|   3.6|         1|
|20918|GO WALK 5-SPARROW...|   4.3|         1|
| 2592|Genuine Leather F...|   3.9|         1|
|14541|Genuine Leather F...|   3.9|         1|
| 7561|Men Na

In [0]:
# Dsiplay number of products per category

flipkart_df.groupBy("maincateg").count().show()

+---------+-----+
|maincateg|count|
+---------+-----+
|     null|   67|
|      Men| 2145|
|    Women| 3032|
+---------+-----+



In [0]:
# Display Average rating per category

flipkart_df.groupBy("maincateg").agg({"Rating": "avg"}).show()

+---------+------------------+
|maincateg|       avg(Rating)|
+---------+------------------+
|     null|             4.225|
|      Men|3.9662970106075246|
|    Women|4.0416582064297915|
+---------+------------------+



In [0]:
# Dsiplay Category with highest average number of reviews

flipkart_df.groupBy("maincateg").agg({"noreviews1": "avg"}) \
    .orderBy("avg(noreviews1)", asc=False).show(1)

+---------+------------------+
|maincateg|   avg(noreviews1)|
+---------+------------------+
|    Women|176.54683377308706|
+---------+------------------+
only showing top 1 row



In [0]:
# Top 5 products with highest price. display product name and price

# Select title and price, order by price descending, limit to top 5
top_5_expensive = flipkart_df.select("title", "actprice1") \
    .orderBy(flipkart_df.actprice1.desc()) \
    .limit(5)

top_5_expensive.show(truncate=False)


+----------------------------------------------------------+---------+
|title                                                     |actprice1|
+----------------------------------------------------------+---------+
|GEL-QUANTUM 360 KNIT Running Shoes For Women  (Blue, Grey)|15999    |
|Women Orange Sports Sandal                                |12999    |
|GT-2000 5 (D) Running Shoes For Women  (Blue)             |10999    |
|Hybrid Astro Running Shoes For Men  (Black)               |10999    |
|DEERUPT RUNNER W Sneakers For Women  (White)              |9999     |
+----------------------------------------------------------+---------+



In [0]:
# Display Min, max, and avg price per category

from pyspark.sql.functions import min, max, avg

# Group by main category and aggregate min, max, and average price
price_stats_by_category = flipkart_df.groupBy("maincateg").agg(
    min("actprice1").alias("Min_Price"),
    max("actprice1").alias("Max_Price"),
    avg("actprice1").alias("Avg_Price")
)

# Show the result sorted by Avg_Price descending (optional)
price_stats_by_category.orderBy("Avg_Price", ascending=False).show(truncate=False)


+---------+---------+---------+------------------+
|maincateg|Min_Price|Max_Price|Avg_Price         |
+---------+---------+---------+------------------+
|null     |499      |5999     |1574.7313432835822|
|Men      |139      |10999    |1455.86013986014  |
|Women    |162      |15999    |1319.7081134564644|
+---------+---------+---------+------------------+



In [0]:
# Display number of nulls in each column

from pyspark.sql.functions import col, sum, when

# Count nulls for each column
null_counts = flipkart_df.select([
    sum(when(col(c).isNull(), 1).otherwise(0)).alias(c)
    for c in flipkart_df.columns
])

null_counts.show()


+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
| id|title|Rating|maincateg|platform|actprice1|norating1|noreviews1|star_5f|star_4f|star_3f|star_2f|star_1f|fulfilled1|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+
|  0|    0|   203|       67|       0|        0|        0|         0|     68|      0|      0|      0|    186|         0|
+---+-----+------+---------+--------+---------+---------+----------+-------+-------+-------+-------+-------+----------+



In [0]:
# calculate and display the category name, number of fulfilled, and unfulfilled products

from pyspark.sql.functions import col, sum, when

# Group by main category and count fulfilled and unfulfilled products
category_fulfillment = flipkart_df.groupBy("maincateg").agg(
    sum(when(col("fulfilled1") == 1, 1).otherwise(0)).alias("Fulfilled_Products"),
    sum(when(col("fulfilled1") == 0, 1).otherwise(0)).alias("Unfulfilled_Products")
)

# Show the result
category_fulfillment.show()


+---------+------------------+--------------------+
|maincateg|Fulfilled_Products|Unfulfilled_Products|
+---------+------------------+--------------------+
|     null|                40|                  27|
|      Men|              1352|                 793|
|    Women|              1778|                1254|
+---------+------------------+--------------------+



In [0]:
# Display Count of products per category

# Group by main category and count the number of products
product_count_by_category = display(flipkart_df.groupBy("maincateg").count())


maincateg,count
null,67
Men,2145
Women,3032


In [0]:
# Display Average rating per category

from pyspark.sql.functions import avg

# Group by category and calculate average rating
avg_rating_by_category = flipkart_df.groupBy("maincateg").agg(
    avg("Rating").alias("Avg_Rating")
)

# Show the result rounded to 2 decimal places
avg_rating_by_category.orderBy("Avg_Rating", ascending=False).show(truncate=False)


+---------+------------------+
|maincateg|Avg_Rating        |
+---------+------------------+
|null     |4.225             |
|Women    |4.0416582064297915|
|Men      |3.9662970106075246|
+---------+------------------+



In [0]:
# Display Category with highest average number of reviews

# Group by category and calculate average number of reviews
avg_reviews_by_category = flipkart_df.groupBy("maincateg").agg(
    avg("noreviews1").alias("Avg_Reviews")
)

# Get the category with the highest average reviews
top_category = avg_reviews_by_category.orderBy("Avg_Reviews", ascending=False).limit(1).show(truncate=False)


+---------+-----------------+
|maincateg|Avg_Reviews      |
+---------+-----------------+
|Men      |759.6554778554779|
+---------+-----------------+



In [0]:
# Display Bar chart of product count per category

# Use Databricks UI to visualize this as a bar chart

product_count_by_category = flipkart_df.groupBy("maincateg").count()

# Display the result (Databricks UI will let you visualize this)
display(product_count_by_category)



maincateg,count
null,67
Men,2145
Women,3032


Databricks visualization. Run in Databricks to view.

In [0]:
# Bar chart of average rating per category

avg_rating_by_category = display(flipkart_df.groupBy("maincateg").agg(
    avg("Rating").alias("Avg_Rating"))
)


maincateg,Avg_Rating
null,4.225
Men,3.9662970106075246
Women,4.0416582064297915


Databricks visualization. Run in Databricks to view.

In [0]:
# Display Bar chart of total number of reviews per category
# Group by category and sum the reviews
total_reviews_by_category = display(flipkart_df.groupBy("maincateg").agg(
    sum("noreviews1").alias("Total_Reviews"))
)

maincateg,Total_Reviews
null,14084
Men,1629461
Women,535290


Databricks visualization. Run in Databricks to view.

In [0]:
# Display product name and 5-star rating for those products with the highest 5-star rating

display(flipkart_df.filter(flipkart_df["star_5f"] > 0).select("title", "star_5f").orderBy("star_5f", ascending=False))

title,star_5f
"Jio-13 Running Shoes For Men (Black, Grey)",151193
"Running Shoes For Men (Silver, Black)",151193
"Running Shoes For Men (Silver, Black)",151173
"Jio-13 Running Shoes For Men (Black, Grey)",151160
"Jio-13 Running Shoes For Men (Red, Green)",151159
"Jio-13 Running Shoes For Men (Red, Green)",101250
VERTIGO BLK RUNNING SHOES For MEN 10 Running Shoes For Men (Black),66369
"Cosko Sports Shoes,Running Shoes,Walking Shoes,Training Shoes, Running Shoes For Men (Navy, Blue)",64987
"Cosko Sports Shoes,Running Shoes,Walking Shoes,Training Shoes, Running Shoes For Men (Navy, Blue)",64987
"Cosko Sports Shoes,Running Shoes,Walking Shoes,Training Shoes, Running Shoes For Men (Navy, Blue)",64981
